## Zero Arb On chain Study 

This study compares the price for a given cow trade with Univ3 trades of the same tokens and same direction within the same block. In particular, it compares the cow price with a)the best available Univ3 price in the block b)the volume weighted average price of univ3 in the block. 

The best available price given by `univ3_max` represents the best price univ3 executed at but does not take into consideration the volumes of these trades. A higher volume trade is expected to fetch a worse price given the impact to the pool so comparing with the vwap price given by `univ3_vwap` allows one to compare the price obtained by cow  to the volume weighted price of the block.  

note that if a block has one univ3 trade, then `univ3_max` = `univ3_vwap` 


outline of process: 

1. COW Trades : Obtain the last COW trades for a given query size.
2. UNI Swaps: Obtain UNI swaps for a query size * 5 (best is to match by timestamp so we have perfectly overlapping uni and cow samples) 
3. Merge COW with UNI on timestamp, token1, token2 : This results in the trades that are overlapping
4. Split the UNI side of dataframe and group by 

### 1. COW Trades

In [625]:
from datastreams.datastream import Streamer

import matplotlib.pyplot as plt
import pandas as pd
import polars as pl

# These commands enlarge the column size of the dataframe so things like 0x... are not truncated
pd.set_option('display.max_columns', None)
pd.set_option('display.expand_frame_repr', False)
pd.set_option('max_colwidth', None)

In [626]:
# instantiate Streamer class. Note that we need two separate streamer classes, otherwise the queries will be overwritten. 
cow_ds = Streamer('https://api.thegraph.com/subgraphs/name/cowprotocol/cow')
#cow_ds2 = Streamer('https://api.thegraph.com/subgraphs/name/cowprotocol/cow')

In [627]:
# DEFINE TIMESTAMP HERE. Timstamp is used for replication quality assurance purposes. Timestamp date: Wed Aug 31 2022 23:00:00 GMT+0000
timestamp =  1672574400

# we set a fixed query size number. The Cow settlements and Uniswap swaps query are multiples larger than this initial query size.
query_size = 5000

#Filter size - We filter trades out that are smaller than $1000 USD size
filter_usd = 100

In [628]:
# query COW schema: trades
trades_fp = cow_ds.queryDict.get('trades')

# trades query path that gets token a -> token b trades
trades_qp = trades_fp(
    first=query_size,
    orderBy='timestamp',
    orderDirection='desc',
    where = {
    'timestamp_gt': timestamp
    }
)

# run query
trades_df = cow_ds.runQuery(trades_qp)

FIELD - trades


In [629]:
print(f'query returned {len(trades_df)} rows')

query returned 5000 rows


In [630]:
# get unique values in trades_df column to verify the query results.
len(trades_df['trades_buyToken_id'].unique())

312

In [631]:
trades_df.head(20)

,trades_id,trades_timestamp,trades_gasPrice,trades_feeAmount,trades_txHash,trades_settlement_id,trades_buyAmount,trades_sellAmount,trades_sellToken_id,trades_buyToken_id,trades_order_id,trades_buyAmountEth,trades_sellAmountEth,trades_buyAmountUsd,trades_sellAmountUsd,endpoint
0,0xbd0afa34cf1ce7521f14cfc5e6cd176bc85b6690f3e9bb3f18177ff0bc2d91867c41a88e599cf947db253c76f4e726fa9afb331b63fc8b7e|0xc8e5cf8ab325f60e305661c787cf14938053e14da3861f2de93f4f6d6df650a9|136,1677489215,17546085577,21625385,0xc8e5cf8ab325f60e305661c787cf14938053e14da3861f2de93f4f6d6df650a9,0xc8e5cf8ab325f60e305661c787cf14938053e14da3861f2de93f4f6d6df650a9,31290352108115149198,10000000000,0xa0b86991c6218b36c1d19d4a2e9eb0ce3606eb48,0xc55126051b22ebb829d00368f4b12bde432de5da,0xbd0afa34cf1ce7521f14cfc5e6cd176bc85b6690f3e9bb3f18177ff0bc2d91867c41a88e599cf947db253c76f4e726fa9afb331b63fc8b7e,0.000000,6.110951,0.000000,10000.000000,cow
1,0x14f5f61f53702cb150db037677ee69212e0231e73fbfb68d2e1e24f6f6fc03c37307955d6a4edc13ce182498f2b5ba4e3585c16f63fc7a81|0xd834bce0c777647ac9888cedd20fb964cfc89cdb72b820eb29a211f3df4153d7|122,1677489083,18786007470,15126860563126448128,0xd834bce0c777647ac9888cedd20fb964cfc89cdb72b820eb29a211f3df4153d7,0xd834bce0c777647ac9888cedd20fb964cfc89cdb72b820eb29a211f3df4153d7,16500000000000000000000,1522622043749214636306,0xaac41ec512808d64625576eddd580e7ea40ef8b2,0xed0439eacf4c4965ae4613d77a5c2efe10e5f183,0x14f5f61f53702cb150db037677ee69212e0231e73fbfb68d2e1e24f6f6fc03c37307955d6a4edc13ce182498f2b5ba4e3585c16f63fc7a81,0.000000,0.000000,0.000000,0.000000,cow
2,0xb9d604e1562ba69184c7f495d77fc6a355e6a2171a77964abfcddeaf323c28b9a136a3abc184bf70de3614822b2a6e6d8df018e563fc7a1c|0x3b6aea7288b91fe9bbfba89f2f7ee452808290e5b9139a724c3486d682ca973a|141,1677488963,17685460249,7304861,0x3b6aea7288b91fe9bbfba89f2f7ee452808290e5b9139a724c3486d682ca973a,0x3b6aea7288b91fe9bbfba89f2f7ee452808290e5b9139a724c3486d682ca973a,779059978130157526455,786103087,0xa0b86991c6218b36c1d19d4a2e9eb0ce3606eb48,0x6b175474e89094c44da98b954eedeac495271d0f,0xb9d604e1562ba69184c7f495d77fc6a355e6a2171a77964abfcddeaf323c28b9a136a3abc184bf70de3614822b2a6e6d8df018e563fc7a1c,0.476080,0.480384,779.059978,786.103087,cow
3,0x9251e5c2a61e2b7584c7a16dfc05671cd50f8514b08d4e447d2e5b11ed9c53009194a2cbb3fc781f295ae38c4c9274736aede8fb63fc8a26|0xa841ee2a69dfa725bab1087f184a789632714b9a409e94c1166ae275c06c1434|119,1677488879,16631818197,5100640856418792448,0xa841ee2a69dfa725bab1087f184a789632714b9a409e94c1166ae275c06c1434,0xa841ee2a69dfa725bab1087f184a789632714b9a409e94c1166ae275c06c1434,141860122404335333,215598381504561302791,0xaf5191b0de278c7286d6c7cc6ab6bb8a73ba2cd6,0xeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeee,0x9251e5c2a61e2b7584c7a16dfc05671cd50f8514b08d4e447d2e5b11ed9c53009194a2cbb3fc781f295ae38c4c9274736aede8fb63fc8a26,0.141860,0.143962,232.140842,235.581119,cow
4,0xb7051051febf1f3c7aeca34d9eece1d53584c9def575f057b9f152889b98f812f6ee1a630bc3ecd8c3fb11520d5952031d7b4c2463fc793a|0x3b4fcfb77f00985ac2b674df8350aef79cf679b4e6377a711683f032fbae1dec|109,1677488783,17740072421,23709624795124903936,0x3b4fcfb77f00985ac2b674df8350aef79cf679b4e6377a711683f032fbae1dec,0x3b4fcfb77f00985ac2b674df8350aef79cf679b4e6377a711683f032fbae1dec,25176680715955121060759,25305475464627336056176,0x96e61422b6a9ba0e068b6c5add4ffabc6a4aae27,0x0f83287ff768d1c1e17a42f44d644d7f22e8ee1d,0xb7051051febf1f3c7aeca34d9eece1d53584c9def575f057b9f152889b98f812f6ee1a630bc3ecd8c3fb11520d5952031d7b4c2463fc793a,0.000000,0.000000,0.000000,0.000000,cow
5,0x330ee01ee9e0d3248c5c7cbb2f0c39886f5bdf84c48ac6d40f2335405d9c244cf6ee1a630bc3ecd8c3fb11520d5952031d7b4c2463fc7921|0x7cc675f290fdacba78d4438300fb6b9a338bfed238d9c7277408bc345b30f8b1|131,1677488747,17412990902,14943866868794100,0x7cc675f290fdacba78d4438300fb6b9a338bfed238d9c7277408bc345b30f8b1,0x7cc675f290fdacba78d4438300fb6b9a338bfed238d9c7277408bc345b30f8b1,742415980233427100000,500000000000000000,0x5e74c9036fb86bd7ecdcb084a0673efc32ea31cb,0x0f83287ff768d1c1e17a42f44d644d7f22e8ee1d,0x330ee01ee

In [632]:
# Set query size large enough to retrieve all tokens in the graph
query_size_token = 10000000

token_fp = cow_ds.queryDict.get('tokens')

# add parameters to the settlements_qp.
token_qp = token_fp(
    first=query_size_token,
)

# run query
token_df = cow_ds.runQuery(token_qp)

FIELD - tokens


In [633]:
# Merge1 on sell token address.  x = sell token 

merged_df = pd.merge(trades_df, token_df, left_on='trades_sellToken_id', right_on='tokens_address')

In [634]:
# Merge2 on buy token address.  y = buy token 
merged_df2 = pd.merge(merged_df, token_df, left_on='trades_buyToken_id', right_on='tokens_address')

In [635]:
merged_df2 = merged_df2.rename(columns={
    "tokens_symbol_x": "sell_token_symbol",
    "tokens_symbol_y":"buy_token_symbol",
    "tokens_decimals_x": "sell_token_decimal", 
    "tokens_decimals_y": "buy_token_decimal"})

In [636]:
# filter necessary columns
cow_df = merged_df2[[
    'trades_id',
    'trades_timestamp', 
    'trades_gasPrice', 
    'trades_feeAmount',                 
    'trades_txHash',                    
    'trades_settlement_id',   
    'trades_sellAmount',
    'sell_token_decimal',
    'trades_buyAmount',   
    'buy_token_decimal',
    'trades_sellToken_id',              
    'trades_buyToken_id',               
    'trades_order_id',                  
    'sell_token_symbol',
    'buy_token_symbol'
    ]]


In [637]:
cow_df.loc[:, 'sell_amount'] = cow_df['trades_sellAmount'] / (10**cow_df['sell_token_decimal'])
cow_df.loc[:, 'buy_amount'] = cow_df['trades_buyAmount'] / (10**cow_df['buy_token_decimal'])


/tmp/ipykernel_699822/1103516907.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  cow_df.loc[:, 'sell_amount'] = cow_df['trades_sellAmount'] / (10**cow_df['sell_token_decimal'])
/tmp/ipykernel_699822/1103516907.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  cow_df.loc[:, 'buy_amount'] = cow_df['trades_buyAmount'] / (10**cow_df['buy_token_decimal'])


In [638]:
# calculate COW price defined sell amount / buy amount
cow_df['cow_sell_buy_ratio'] = cow_df['sell_amount'] / cow_df['buy_amount']

/tmp/ipykernel_699822/3636551783.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  cow_df['cow_sell_buy_ratio'] = cow_df['sell_amount'] / cow_df['buy_amount']


In [639]:
cow_df.shape

(5000, 18)

In [640]:
cow_df.dtypes

trades_id               object
trades_timestamp         int64
trades_gasPrice          int64
trades_feeAmount        object
trades_txHash           object
trades_settlement_id    object
trades_sellAmount       object
sell_token_decimal       int64
trades_buyAmount        object
buy_token_decimal        int64
trades_sellToken_id     object
trades_buyToken_id      object
trades_order_id         object
sell_token_symbol       object
buy_token_symbol        object
sell_amount             object
buy_amount              object
cow_sell_buy_ratio      object
dtype: object

In [641]:
cow_df.head(5)

,trades_id,trades_timestamp,trades_gasPrice,trades_feeAmount,trades_txHash,trades_settlement_id,trades_sellAmount,sell_token_decimal,trades_buyAmount,buy_token_decimal,trades_sellToken_id,trades_buyToken_id,trades_order_id,sell_token_symbol,buy_token_symbol,sell_amount,buy_amount,cow_sell_buy_ratio
0,0xbd0afa34cf1ce7521f14cfc5e6cd176bc85b6690f3e9bb3f18177ff0bc2d91867c41a88e599cf947db253c76f4e726fa9afb331b63fc8b7e|0xc8e5cf8ab325f60e305661c787cf14938053e14da3861f2de93f4f6d6df650a9|136,1677489215,17546085577,21625385,0xc8e5cf8ab325f60e305661c787cf14938053e14da3861f2de93f4f6d6df650a9,0xc8e5cf8ab325f60e305661c787cf14938053e14da3861f2de93f4f6d6df650a9,10000000000,6,31290352108115149198,18,0xa0b86991c6218b36c1d19d4a2e9eb0ce3606eb48,0xc55126051b22ebb829d00368f4b12bde432de5da,0xbd0afa34cf1ce7521f14cfc5e6cd176bc85b6690f3e9bb3f18177ff0bc2d91867c41a88e599cf947db253c76f4e726fa9afb331b63fc8b7e,USDC,BTRFLY,10000.0,31.290352,319.587327
1,0x3ec6b050d7c8282bb43562beccbeb57a6584db51726f10c9790809a368d526863f6e4667fec54733fb3b15993b9ce96b69b65cb363fc3a72|0x6724622a4ae25c451b6c7d72b062f052143f47b59fee00b0d0ef2ae4262739a7|6,1677468455,18856720780,5028307,0x6724622a4ae25c451b6c7d72b062f052143f47b59fee00b0d0ef2ae4262739a7,0x6724622a4ae25c451b6c7d72b062f052143f47b59fee00b0d0ef2ae4262739a7,350000000,6,1029171942374219412,18,0xa0b86991c6218b36c1d19d4a2e9eb0ce3606eb48,0xc55126051b22ebb829d00368f4b12bde432de5da,0x3ec6b050d7c8282bb43562beccbeb57a6584db51726f10c9790809a368d526863f6e4667fec54733fb3b15993b9ce96b69b65cb363fc3a72,USDC,BTRFLY,350.0,1.029172,340.079228
2,0xc3c45dbdbcbb8556f01951dc5f45a76fa32a59fd32b54aa71f8080b3d459f939243706e13823949dca2283ce5607f618d9d17c3163f91d43|0x86f15c1f2b6591e3ada5d864d0a1504f349565898abc22c508645e1f82d3cadf|142,1677268607,25062272376,30830816,0x86f15c1f2b6591e3ada5d864d0a1504f349565898abc22c508645e1f82d3cadf,0x86f15c1f2b6591e3ada5d864d0a1504f349565898abc22c508645e1f82d3cadf,3000000000,6,8522001559191804345,18,0xa0b86991c6218b36c1d19d4a2e9eb0ce3606eb48,0xc55126051b22ebb829d00368f4b12bde432de5da,0xc3c45dbdbcbb8556f01951dc5f45a76fa32a59fd32b54aa71f8080b3d459f939243706e13823949dca2283ce5607f618d9d17c3163f91d43,USDC,BTRFLY,3000.0,8.522002,352.029975
3,0x9fa1be474850f57f05c7fd4aa7181dd84f3299a02175e2783aea55ec46cc414c003127b115296ecef3bea304fd3f7eb3dc4172ca63f8294b|0xea1f482059c9444237fdfd48b258427ead828b86a01c0a08164d9d8af6a1a7e6|179,1677201983,29330229855,11241095,0xea1f482059c9444237fdfd48b258427ead828b86a01c0a08164d9d8af6a1a7e6,0xea1f482059c9444237fdfd48b258427ead828b86a01c0a08164d9d8af6a1a7e6,113351611,6,270401236594523802,18,0xa0b86991c6218b36c1d19d4a2e9eb0ce3606eb48,0xc55126051b22ebb829d00368f4b12bde432de5da,0x9fa1be474850f57f05c7fd4aa7181dd84f3299a02175e2783aea55ec46cc414c003127b115296ecef3bea304fd3f7eb3dc4172ca63f8294b,USDC,BTRFLY,113.351611,0.270401,419.197828
4,0x678ff17da507aeb835e0e82c7da3d8d259616daacc0c4d4ea736232d05f5bda4e445e51dd3ace310a0ba033addef0b02097bf08b63f7d32e|0x742651361c7f6ec4146495f4b6de424eebc8e4b15ebe6903f48bdb91ae7a87f8|129,1677184115,39546309230,26407408,0x742651361c7f6ec4146495f4b6de424eebc8e4b15ebe6903f48bdb91ae7a87f8,0x742651361c7f6ec4146495f4b6de424eebc8e4b15ebe6903f48bdb91ae7a87f8,5210652400,6,13565146042005061017,18,0xa0b86991c6218b36c1d19d4a2e9eb0ce3606eb48,0xc55126051b22ebb829d00368f4b12bde432de5da,0x678ff17da507aeb835e0e82c7da3d8d259616daacc0c4d4ea736232d05f5bda4e445e51dd3ace310a0ba033addef0b02097bf08b63f7d32e,USDC,BTRFLY,5210.6524,13.565146,384.120627


### 2. Uniswap V3 Swaps

In [642]:
# instantiate Streamer object. 
# Note - unlike the cow queries, univ3 does not require multiple streamer instantations because the swaps field path is reset each iteration. 
# If the Cow queries were updated to use the same method, we could use the same streamer object for all queries.
univ3_ds = Streamer('https://api.thegraph.com/subgraphs/name/messari/uniswap-v3-ethereum')

In [643]:
# get a query field path from the query dictionary which is automatically populated in the Streamer object
uni_swaps_fp = univ3_ds.queryDict.get('swaps')

In [644]:
uni_swaps_qp = uni_swaps_fp(
        first=query_size * 5,
        orderBy='timestamp',
        orderDirection='desc',
        where = {'timestamp_gt': timestamp, 'amountInUSD_gt': filter_usd, 'amountOutUSD_gt': filter_usd} 
        )

# run query
uni_swaps_df = univ3_ds.runQuery(uni_swaps_qp)
    

FIELD - swaps


In [645]:
uni_swaps_df.shape

(25000, 16)

In [646]:
uni_swaps_df.dtypes

swaps_id               object
swaps_hash             object
swaps_logIndex          int64
swaps_protocol_id      object
swaps_to               object
swaps_from             object
swaps_blockNumber       int64
swaps_timestamp         int64
swaps_tokenIn_id       object
swaps_amountIn         object
swaps_amountInUSD     float64
swaps_tokenOut_id      object
swaps_amountOut        object
swaps_amountOutUSD    float64
swaps_pool_id          object
endpoint               object
dtype: object

In [647]:
uni_swaps_df.head(5)

,swaps_id,swaps_hash,swaps_logIndex,swaps_protocol_id,swaps_to,swaps_from,swaps_blockNumber,swaps_timestamp,swaps_tokenIn_id,swaps_amountIn,swaps_amountInUSD,swaps_tokenOut_id,swaps_amountOut,swaps_amountOutUSD,swaps_pool_id,endpoint
0,0xf9ace0f79c10ab0fff61c888659ecf01c69ec2d8d8e8f7f158ec3707c3120deb-167,0xf9ace0f79c10ab0fff61c888659ecf01c69ec2d8d8e8f7f158ec3707c3120deb,167,0x1f98431c8ad98523631ae4a59f267346ea31f984,0xbaf7c4c1d3f947d5404d317df4227551b33b6b3d,0xbaf7c4c1d3f947d5404d317df4227551b33b6b3d,16718585,1677489407,0xa0b86991c6218b36c1d19d4a2e9eb0ce3606eb48,8543029671,8543.029671,0x2b591e99afe9f32eaa6214f7b7629768c40eeb39,12790661865420,8506.698449,0x69d91b94f0aaf8e8a2586909fa77a5c2c89818d5,uniswap-v3-ethereum
1,0xf9ace0f79c10ab0fff61c888659ecf01c69ec2d8d8e8f7f158ec3707c3120deb-164,0xf9ace0f79c10ab0fff61c888659ecf01c69ec2d8d8e8f7f158ec3707c3120deb,164,0x1f98431c8ad98523631ae4a59f267346ea31f984,0xdef1c0ded9bec7f1a1670819833240f027b25eff,0xbaf7c4c1d3f947d5404d317df4227551b33b6b3d,16718585,1677489407,0xc02aaa39b223fe8d0a0e5c4f27ead9083c756cc2,5225000000000000000,8550.224527,0xa0b86991c6218b36c1d19d4a2e9eb0ce3606eb48,8543029671,8543.029671,0xe0554a476a092703abdb3ef35c80e0d76d32939f,uniswap-v3-ethereum
2,0xf9ace0f79c10ab0fff61c888659ecf01c69ec2d8d8e8f7f158ec3707c3120deb-161,0xf9ace0f79c10ab0fff61c888659ecf01c69ec2d8d8e8f7f158ec3707c3120deb,161,0x1f98431c8ad98523631ae4a59f267346ea31f984,0xbaf7c4c1d3f947d5404d317df4227551b33b6b3d,0xbaf7c4c1d3f947d5404d317df4227551b33b6b3d,16718585,1677489407,0xc02aaa39b223fe8d0a0e5c4f27ead9083c756cc2,5775000000000000000,9450.248161,0x2b591e99afe9f32eaa6214f7b7629768c40eeb39,14138048595787,9402.806307,0x9e0905249ceefffb9605e034b534544684a58be6,uniswap-v3-ethereum
3,0xed76fbb38d76dc34c45a10ceb1975e3595258155e09e483b306af838408c7c2d-360,0xed76fbb38d76dc34c45a10ceb1975e3595258155e09e483b306af838408c7c2d,360,0x1f98431c8ad98523631ae4a59f267346ea31f984,0x7122db0ebe4eb9b434a9f2ffe6760bc03bfbd0e0,0xeb2609ab5ca27dbca7f05d271bc539357c77f8c9,16718585,1677489407,0xa0b86991c6218b36c1d19d4a2e9eb0ce3606eb48,8500149215,8500.149215,0x1abaea1f7c830bd89acc67ec4af516284b1bc33c,8012649623,8493.957046,0x95dbb3c7546f22bce375900abfdd64a4e5bd73d6,uniswap-v3-ethereum
4,0xcc4ac5e21eb5cf3901ec4b2bc9e7332862d7d5fb75c88e0fde428240e916f0fd-289,0xcc4ac5e21eb5cf3901ec4b2bc9e7332862d7d5fb75c88e0fde428240e916f0fd,289,0x1f98431c8ad98523631ae4a59f267346ea31f984,0xdef171fe48cf0115b1d80b88dc8eab59176fee57,0x3213977900a71e183818472e795c76af8cbc3a3e,16718585,1677489407,0xc02aaa39b223fe8d0a0e5c4f27ead9083c756cc2,463513370889166616,758.496343,0xf59257e961883636290411c11ec5ae622d19455e,202935911133,750.042872,0xb386c1d831eed803f5e8f274a59c91c4c22eeac0,uniswap-v3-ethereum


In [649]:
# Set query size large enough to retrieve all tokens in the graph
query_size_token = 10000

uni_token_fp = univ3_ds.queryDict.get('tokens')

# add parameters to the settlements_qp.
uni_token_qp = uni_token_fp(
    first=query_size_token,
)

# run query
uni_token_df = univ3_ds.runQuery(uni_token_qp)

FIELD - tokens


In [650]:
uni_token_df.shape

(7097, 7)

In [651]:
uni_token_df.dtypes

tokens_id                       object
tokens_name                     object
tokens_symbol                   object
tokens_decimals                  int64
tokens_lastPriceUSD            float64
tokens_lastPriceBlockNumber      int64
endpoint                        object
dtype: object

In [652]:
# Merge1 on tokenIN address.   x is in 

merge1_uni_df = pd.merge(uni_swaps_df, uni_token_df, left_on='swaps_tokenIn_id', right_on='tokens_id')

In [653]:
# Merge2 on tokenOUT address.  y is out 

merge2_uni_df = pd.merge(merge1_uni_df, uni_token_df, left_on='swaps_tokenOut_id', right_on='tokens_id')

In [654]:
merge2_uni_df.dtypes

swaps_id                          object
swaps_hash                        object
swaps_logIndex                     int64
swaps_protocol_id                 object
swaps_to                          object
swaps_from                        object
swaps_blockNumber                  int64
swaps_timestamp                    int64
swaps_tokenIn_id                  object
swaps_amountIn                    object
swaps_amountInUSD                float64
swaps_tokenOut_id                 object
swaps_amountOut                   object
swaps_amountOutUSD               float64
swaps_pool_id                     object
endpoint_x                        object
tokens_id_x                       object
tokens_name_x                     object
tokens_symbol_x                   object
tokens_decimals_x                  int64
tokens_lastPriceUSD_x            float64
tokens_lastPriceBlockNumber_x      int64
endpoint_y                        object
tokens_id_y                       object
tokens_name_y   

In [655]:
merge2_uni_df = merge2_uni_df.rename(columns={
    "tokens_symbol_x": "tokenIn_symbol",
    "tokens_symbol_y":"tokenOut_symbol",
    "tokens_decimals_x": "tokenIn_decimal", 
    "tokens_decimals_y": "tokenOut_decimal"})

In [656]:
# filter necessary columns
uni_df = merge2_uni_df[[
    'swaps_id',
    'swaps_hash',
    'swaps_protocol_id',
    'swaps_blockNumber',
    'swaps_timestamp',
    'swaps_tokenIn_id',
    'tokenIn_symbol',
    'tokenIn_decimal',
    'swaps_amountIn',
    'swaps_tokenOut_id',
    'tokenOut_symbol',
    'tokenOut_decimal',
    'swaps_amountOut',
    'swaps_pool_id'
    ]]

In [657]:
uni_df.head(5)

,swaps_id,swaps_hash,swaps_protocol_id,swaps_blockNumber,swaps_timestamp,swaps_tokenIn_id,tokenIn_symbol,tokenIn_decimal,swaps_amountIn,swaps_tokenOut_id,tokenOut_symbol,tokenOut_decimal,swaps_amountOut,swaps_pool_id
0,0xf9ace0f79c10ab0fff61c888659ecf01c69ec2d8d8e8f7f158ec3707c3120deb-167,0xf9ace0f79c10ab0fff61c888659ecf01c69ec2d8d8e8f7f158ec3707c3120deb,0x1f98431c8ad98523631ae4a59f267346ea31f984,16718585,1677489407,0xa0b86991c6218b36c1d19d4a2e9eb0ce3606eb48,USDC,6,8543029671,0x2b591e99afe9f32eaa6214f7b7629768c40eeb39,HEX,8,12790661865420,0x69d91b94f0aaf8e8a2586909fa77a5c2c89818d5
1,0x35b7f221c45070fa0ebb63d30a7376b48a7731e5b43346687eac6862a3fb4e2c-5,0x35b7f221c45070fa0ebb63d30a7376b48a7731e5b43346687eac6862a3fb4e2c,0x1f98431c8ad98523631ae4a59f267346ea31f984,16718556,1677489059,0xa0b86991c6218b36c1d19d4a2e9eb0ce3606eb48,USDC,6,2000000000,0x2b591e99afe9f32eaa6214f7b7629768c40eeb39,HEX,8,2996122355248,0x69d91b94f0aaf8e8a2586909fa77a5c2c89818d5
2,0x1ab168e35fa003cf6b4fe815cf426591501b97382cd9783f44644bff4ae06015-182,0x1ab168e35fa003cf6b4fe815cf426591501b97382cd9783f44644bff4ae06015,0x1f98431c8ad98523631ae4a59f267346ea31f984,16718537,1677488819,0xa0b86991c6218b36c1d19d4a2e9eb0ce3606eb48,USDC,6,16477896379,0x2b591e99afe9f32eaa6214f7b7629768c40eeb39,HEX,8,24632435994139,0x69d91b94f0aaf8e8a2586909fa77a5c2c89818d5
3,0xc44752eb69ec0918ec57995c16cde97852a7ae59b6bf5c57ca3fea005948d4af-82,0xc44752eb69ec0918ec57995c16cde97852a7ae59b6bf5c57ca3fea005948d4af,0x1f98431c8ad98523631ae4a59f267346ea31f984,16718524,1677488663,0xa0b86991c6218b36c1d19d4a2e9eb0ce3606eb48,USDC,6,14720947379,0x2b591e99afe9f32eaa6214f7b7629768c40eeb39,HEX,8,22107318478989,0x69d91b94f0aaf8e8a2586909fa77a5c2c89818d5
4,0x8ac5c32348fa7da2a7273a0aa9e5b22ae4cd690359b33b55401602138de740b0-193,0x8ac5c32348fa7da2a7273a0aa9e5b22ae4cd690359b33b55401602138de740b0,0x1f98431c8ad98523631ae4a59f267346ea31f984,16718509,1677488483,0xa0b86991c6218b36c1d19d4a2e9eb0ce3606eb48,USDC,6,3422061260,0x2b591e99afe9f32eaa6214f7b7629768c40eeb39,HEX,8,5105024913922,0x69d91b94f0aaf8e8a2586909fa77a5c2c89818d5


In [658]:
# Get amounts in a uniform decimal 
uni_df.loc[:, 'univ3_In_amount'] = uni_df['swaps_amountIn'] / (10**uni_df['tokenIn_decimal'])
uni_df.loc[:, 'univ3_Out_amount'] = uni_df['swaps_amountOut'] / (10**uni_df['tokenOut_decimal'])

/tmp/ipykernel_699822/2168625794.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  uni_df.loc[:, 'univ3_In_amount'] = uni_df['swaps_amountIn'] / (10**uni_df['tokenIn_decimal'])
/tmp/ipykernel_699822/2168625794.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  uni_df.loc[:, 'univ3_Out_amount'] = uni_df['swaps_amountOut'] / (10**uni_df['tokenOut_decimal'])


In [659]:
# calculate UNI swap price defined In amount / Out amount
uni_df['uni_in_out_ratio'] = uni_df['univ3_In_amount'] / uni_df['univ3_Out_amount']

/tmp/ipykernel_699822/1576826904.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  uni_df['uni_in_out_ratio'] = uni_df['univ3_In_amount'] / uni_df['univ3_Out_amount']


In [660]:
uni_df.head(5)

,swaps_id,swaps_hash,swaps_protocol_id,swaps_blockNumber,swaps_timestamp,swaps_tokenIn_id,tokenIn_symbol,tokenIn_decimal,swaps_amountIn,swaps_tokenOut_id,tokenOut_symbol,tokenOut_decimal,swaps_amountOut,swaps_pool_id,univ3_In_amount,univ3_Out_amount,uni_in_out_ratio
0,0xf9ace0f79c10ab0fff61c888659ecf01c69ec2d8d8e8f7f158ec3707c3120deb-167,0xf9ace0f79c10ab0fff61c888659ecf01c69ec2d8d8e8f7f158ec3707c3120deb,0x1f98431c8ad98523631ae4a59f267346ea31f984,16718585,1677489407,0xa0b86991c6218b36c1d19d4a2e9eb0ce3606eb48,USDC,6,8543029671,0x2b591e99afe9f32eaa6214f7b7629768c40eeb39,HEX,8,12790661865420,0x69d91b94f0aaf8e8a2586909fa77a5c2c89818d5,8543.029671,127906.618654,0.066791
1,0x35b7f221c45070fa0ebb63d30a7376b48a7731e5b43346687eac6862a3fb4e2c-5,0x35b7f221c45070fa0ebb63d30a7376b48a7731e5b43346687eac6862a3fb4e2c,0x1f98431c8ad98523631ae4a59f267346ea31f984,16718556,1677489059,0xa0b86991c6218b36c1d19d4a2e9eb0ce3606eb48,USDC,6,2000000000,0x2b591e99afe9f32eaa6214f7b7629768c40eeb39,HEX,8,2996122355248,0x69d91b94f0aaf8e8a2586909fa77a5c2c89818d5,2000.0,29961.223552,0.066753
2,0x1ab168e35fa003cf6b4fe815cf426591501b97382cd9783f44644bff4ae06015-182,0x1ab168e35fa003cf6b4fe815cf426591501b97382cd9783f44644bff4ae06015,0x1f98431c8ad98523631ae4a59f267346ea31f984,16718537,1677488819,0xa0b86991c6218b36c1d19d4a2e9eb0ce3606eb48,USDC,6,16477896379,0x2b591e99afe9f32eaa6214f7b7629768c40eeb39,HEX,8,24632435994139,0x69d91b94f0aaf8e8a2586909fa77a5c2c89818d5,16477.896379,246324.359941,0.066895
3,0xc44752eb69ec0918ec57995c16cde97852a7ae59b6bf5c57ca3fea005948d4af-82,0xc44752eb69ec0918ec57995c16cde97852a7ae59b6bf5c57ca3fea005948d4af,0x1f98431c8ad98523631ae4a59f267346ea31f984,16718524,1677488663,0xa0b86991c6218b36c1d19d4a2e9eb0ce3606eb48,USDC,6,14720947379,0x2b591e99afe9f32eaa6214f7b7629768c40eeb39,HEX,8,22107318478989,0x69d91b94f0aaf8e8a2586909fa77a5c2c89818d5,14720.947379,221073.18479,0.066589
4,0x8ac5c32348fa7da2a7273a0aa9e5b22ae4cd690359b33b55401602138de740b0-193,0x8ac5c32348fa7da2a7273a0aa9e5b22ae4cd690359b33b55401602138de740b0,0x1f98431c8ad98523631ae4a59f267346ea31f984,16718509,1677488483,0xa0b86991c6218b36c1d19d4a2e9eb0ce3606eb48,USDC,6,3422061260,0x2b591e99afe9f32eaa6214f7b7629768c40eeb39,HEX,8,5105024913922,0x69d91b94f0aaf8e8a2586909fa77a5c2c89818d5,3422.06126,51050.249139,0.067033


### 3. Univ3 : Block Max  & VWAP calculations

Notes: 

- In cow, the sell and buy are from user's point of view. In Uni, the in and out are from pool point of view. 
so sell = in and buy = out 

- since they are all sell/buy then we can compare if cow>uni it means that cow price was better. The higher the better. 


step1: 
step2:
step3: 

#### Step1: Get only UNIV3 trades that match with COW 

In [689]:
# merge trades and swaps on timestamp value, tokens, and direction of trade. We use outer join because we want to keep all trades and swaps data and backfill swap values
cow_uni_df = pd.merge(cow_df, uni_df, left_on=['trades_timestamp','trades_sellToken_id','trades_buyToken_id'], right_on=['swaps_timestamp','swaps_tokenIn_id','swaps_tokenOut_id'])

In [690]:
cow_uni_df.sort_values(by='trades_timestamp', ascending=False, inplace=True)

In [691]:
cow_uni_df.shape

(120, 35)

In [692]:
cow_uni_df.dtypes

trades_id               object
trades_timestamp         int64
trades_gasPrice          int64
trades_feeAmount        object
trades_txHash           object
trades_settlement_id    object
trades_sellAmount       object
sell_token_decimal       int64
trades_buyAmount        object
buy_token_decimal        int64
trades_sellToken_id     object
trades_buyToken_id      object
trades_order_id         object
sell_token_symbol       object
buy_token_symbol        object
sell_amount             object
buy_amount              object
cow_sell_buy_ratio      object
swaps_id                object
swaps_hash              object
swaps_protocol_id       object
swaps_blockNumber        int64
swaps_timestamp          int64
swaps_tokenIn_id        object
tokenIn_symbol          object
tokenIn_decimal          int64
swaps_amountIn          object
swaps_tokenOut_id       object
tokenOut_symbol         object
tokenOut_decimal         int64
swaps_amountOut         object
swaps_pool_id           object
univ3_In

In [693]:
cow_uni_df.head(5)

,trades_id,trades_timestamp,trades_gasPrice,trades_feeAmount,trades_txHash,trades_settlement_id,trades_sellAmount,sell_token_decimal,trades_buyAmount,buy_token_decimal,trades_sellToken_id,trades_buyToken_id,trades_order_id,sell_token_symbol,buy_token_symbol,sell_amount,buy_amount,cow_sell_buy_ratio,swaps_id,swaps_hash,swaps_protocol_id,swaps_blockNumber,swaps_timestamp,swaps_tokenIn_id,tokenIn_symbol,tokenIn_decimal,swaps_amountIn,swaps_tokenOut_id,tokenOut_symbol,tokenOut_decimal,swaps_amountOut,swaps_pool_id,univ3_In_amount,univ3_Out_amount,uni_in_out_ratio
98,0x701a0176715a23bd53609b012f7bd58b67d503087c531df3d0d275cf6780820acbe203901e2f3ca910558efba39a48af89e3c55863fc73be|0xb9d34721a0da9ec11019805740abe5304f16fec3087bde92d2876788b0d544bc|121,1677487391,17033197478,10728549012072273920,0xb9d34721a0da9ec11019805740abe5304f16fec3087bde92d2876788b0d544bc,0xb9d34721a0da9ec11019805740abe5304f16fec3087bde92d2876788b0d544bc,80140774800720310238916,18,67751282514,6,0x5283d291dbcf85356a21ba090e6db59121208b44,0xa0b86991c6218b36c1d19d4a2e9eb0ce3606eb48,0x701a0176715a23bd53609b012f7bd58b67d503087c531df3d0d275cf6780820acbe203901e2f3ca910558efba39a48af89e3c55863fc73be,BLUR,USDC,80140.774801,67751.282514,1.182867,0xb9d34721a0da9ec11019805740abe5304f16fec3087bde92d2876788b0d544bc-234,0xb9d34721a0da9ec11019805740abe5304f16fec3087bde92d2876788b0d544bc,0x1f98431c8ad98523631ae4a59f267346ea31f984,16718418,1677487391,0x5283d291dbcf85356a21ba090e6db59121208b44,BLUR,18,8013004625170824296499,0xa0b86991c6218b36c1d19d4a2e9eb0ce3606eb48,USDC,6,6768900881,0x92ab871abb9d567aa276b2ce58d0203d84e0181e,8013.004625,6768.900881,1.183797
0,0x80fcb44508ad58cc156422124f63ed9a9eb452030c45a20d07bee14e8727508f6d4ac258367dc87b219cd6025c6624199823528a63fd1718|0x472150c620c0773767e6c5d3e9cd2a92cf7e8eb2001fbea5a74c6b86e5be5421|176,1677484151,21094309984,0,0x472150c620c0773767e6c5d3e9cd2a92cf7e8eb2001fbea5a74c6b86e5be5421,0x472150c620c0773767e6c5d3e9cd2a92cf7e8eb2001fbea5a74c6b86e5be5421,30514028354,6,30518143245082152756689,18,0xdac17f958d2ee523a2206206994597c13d831ec7,0x6b175474e89094c44da98b954eedeac495271d0f,0x80fcb44508ad58cc156422124f63ed9a9eb452030c45a20d07bee14e8727508f6d4ac258367dc87b219cd6025c6624199823528a63fd1718,USDT,DAI,30514.028354,30518.143245,0.999865,0x472150c620c0773767e6c5d3e9cd2a92cf7e8eb2001fbea5a74c6b86e5be5421-382,0x472150c620c0773767e6c5d3e9cd2a92cf7e8eb2001fbea5a74c6b86e5be5421,0x1f98431c8ad98523631ae4a59f267346ea31f984,16718150,1677484151,0xdac17f958d2ee523a2206206994597c13d831ec7,USDT,6,30510612544,0x6b175474e89094c44da98b954eedeac495271d0f,DAI,18,30518143256958443375976,0x48da0965ab2d2cbf1c17c09cfb5cbe67ad5b1406,30510.612544,30518.143257,0.999753
65,0xb73d730b3fb39d57ecb8fc58ed443fc1e7d95dff0a15e47ce49fdc020f7631fe36cc7b13029b5dee4034745fb4f24034f3f2ffc663fc7602|0x3df3c991cf0dc85a69c833862c66114a60a1e269d5f3a9ef745a41d16c76fbd2|92,1677483707,15858459108,3255068885491244032,0x3df3c991cf0dc85a69c833862c66114a60a1e269d5f3a9ef745a41d16c76fbd2,0x3df3c991cf0dc85a69c833862c66114a60a1e269d5f3a9ef745a41d16c76fbd2,6161085427427419246920,18,12823960269097420856,18,0xc0c293ce456ff0ed870add98a0828dd4d2903dbf,0xc02aaa39b223fe8d0a0e5c4f27ead9083c756cc2,0xb73d730b3fb39d57ecb8fc58ed443fc1e7d95dff0a15e47ce49fdc020f7631fe36cc7b13029b5dee4034745fb4f24034f3f2ffc663fc7602,AURA,WETH,6161.085427,12.82396,480.435474,0x3df3c991cf0dc85a69c833862c66114a60a1e269d5f3a9ef745a41d16c76fbd2-184,0x3df3c991cf0dc85a69c833862c66114a60a1e269d5f3a9ef745a41d16c76fbd2,0x1f98431c8ad98523631ae4a59f267346ea31f984,16718114,1677483707,0xc0c293ce456ff0ed870add98a0828dd4d2903dbf,AURA,18,1847349107562578400866,0xc02aaa39b223fe8d0a0e5c4f27ead9083c756cc2,WETH,18,3849402145926979227,0x4be410e2ff6a5f1718ada572afa9e8d26537242b,1847.349108,3.849402,479.90546
104,0x4f932d189954a3ed440d52840b058cd3da6465d4584bd9db7448aa792566a8cdbdc3922244608bdb27f619616e6aded5cc0c4e6a63fc61b8|0x151ed3ef9aabdbfeaa7d679deed658af23c0daf949d99a0542dcc819464090fc|161,1677482771,14915399317,13175076080,0x151ed3ef9aabdbf

In [771]:
# Validate grouping 

cow_uni_df_group_test = cow_uni_df.groupby(['trades_timestamp', 'trades_sellToken_id', 'trades_buyToken_id'])
group_sizes = grouped_df_test.size()
print(group_sizes)

swaps_timestamp  swaps_tokenIn_id                            swaps_tokenOut_id                         
1677410507       0xa0b86991c6218b36c1d19d4a2e9eb0ce3606eb48  0x30d20208d987713f46dfd34ef128bb16c404d10f    1
1677415223       0x375abb85c329753b1ba849a601438ae77eec9893  0xc02aaa39b223fe8d0a0e5c4f27ead9083c756cc2    1
1677416447       0x2b591e99afe9f32eaa6214f7b7629768c40eeb39  0xa0b86991c6218b36c1d19d4a2e9eb0ce3606eb48    1
1677418679       0xc02aaa39b223fe8d0a0e5c4f27ead9083c756cc2  0xdac17f958d2ee523a2206206994597c13d831ec7    2
1677418895       0x5283d291dbcf85356a21ba090e6db59121208b44  0xa0b86991c6218b36c1d19d4a2e9eb0ce3606eb48    1
                                                                                                          ..
1677482663       0xa0b86991c6218b36c1d19d4a2e9eb0ce3606eb48  0xc02aaa39b223fe8d0a0e5c4f27ead9083c756cc2    1
1677482771       0x2b591e99afe9f32eaa6214f7b7629768c40eeb39  0xa0b86991c6218b36c1d19d4a2e9eb0ce3606eb48    1
1677483707       0xc0c29

#### Step 2: Group trades in same block if more than one univ3 swap for the same cow trade. 

In [694]:
# Split the merged cow_uni_df into just the univ3 side
uni_temp_df = cow_uni_df.loc[:, 'swaps_id':]

In [695]:
uni_temp_df.shape

(120, 17)

In [696]:
uni_temp_df.dtypes

swaps_id             object
swaps_hash           object
swaps_protocol_id    object
swaps_blockNumber     int64
swaps_timestamp       int64
swaps_tokenIn_id     object
tokenIn_symbol       object
tokenIn_decimal       int64
swaps_amountIn       object
swaps_tokenOut_id    object
tokenOut_symbol      object
tokenOut_decimal      int64
swaps_amountOut      object
swaps_pool_id        object
univ3_In_amount      object
univ3_Out_amount     object
uni_in_out_ratio     object
dtype: object

In [697]:
uni_temp_df.head(5)

,swaps_id,swaps_hash,swaps_protocol_id,swaps_blockNumber,swaps_timestamp,swaps_tokenIn_id,tokenIn_symbol,tokenIn_decimal,swaps_amountIn,swaps_tokenOut_id,tokenOut_symbol,tokenOut_decimal,swaps_amountOut,swaps_pool_id,univ3_In_amount,univ3_Out_amount,uni_in_out_ratio
98,0xb9d34721a0da9ec11019805740abe5304f16fec3087bde92d2876788b0d544bc-234,0xb9d34721a0da9ec11019805740abe5304f16fec3087bde92d2876788b0d544bc,0x1f98431c8ad98523631ae4a59f267346ea31f984,16718418,1677487391,0x5283d291dbcf85356a21ba090e6db59121208b44,BLUR,18,8013004625170824296499,0xa0b86991c6218b36c1d19d4a2e9eb0ce3606eb48,USDC,6,6768900881,0x92ab871abb9d567aa276b2ce58d0203d84e0181e,8013.004625,6768.900881,1.183797
0,0x472150c620c0773767e6c5d3e9cd2a92cf7e8eb2001fbea5a74c6b86e5be5421-382,0x472150c620c0773767e6c5d3e9cd2a92cf7e8eb2001fbea5a74c6b86e5be5421,0x1f98431c8ad98523631ae4a59f267346ea31f984,16718150,1677484151,0xdac17f958d2ee523a2206206994597c13d831ec7,USDT,6,30510612544,0x6b175474e89094c44da98b954eedeac495271d0f,DAI,18,30518143256958443375976,0x48da0965ab2d2cbf1c17c09cfb5cbe67ad5b1406,30510.612544,30518.143257,0.999753
65,0x3df3c991cf0dc85a69c833862c66114a60a1e269d5f3a9ef745a41d16c76fbd2-184,0x3df3c991cf0dc85a69c833862c66114a60a1e269d5f3a9ef745a41d16c76fbd2,0x1f98431c8ad98523631ae4a59f267346ea31f984,16718114,1677483707,0xc0c293ce456ff0ed870add98a0828dd4d2903dbf,AURA,18,1847349107562578400866,0xc02aaa39b223fe8d0a0e5c4f27ead9083c756cc2,WETH,18,3849402145926979227,0x4be410e2ff6a5f1718ada572afa9e8d26537242b,1847.349108,3.849402,479.90546
104,0x151ed3ef9aabdbfeaa7d679deed658af23c0daf949d99a0542dcc819464090fc-335,0x151ed3ef9aabdbfeaa7d679deed658af23c0daf949d99a0542dcc819464090fc,0x1f98431c8ad98523631ae4a59f267346ea31f984,16718037,1677482771,0x2b591e99afe9f32eaa6214f7b7629768c40eeb39,HEX,8,15217815486823,0xa0b86991c6218b36c1d19d4a2e9eb0ce3606eb48,USDC,6,10201441775,0x69d91b94f0aaf8e8a2586909fa77a5c2c89818d5,152178.154868,10201.441775,14.917318
48,0x3f03fb68fe311cfe4d9d956115b27a84a54e592899fabc203d7a2a346d2855ba-206,0x3f03fb68fe311cfe4d9d956115b27a84a54e592899fabc203d7a2a346d2855ba,0x1f98431c8ad98523631ae4a59f267346ea31f984,16718028,1677482663,0xa0b86991c6218b36c1d19d4a2e9eb0ce3606eb48,USDC,6,7596942337,0xc02aaa39b223fe8d0a0e5c4f27ead9083c756cc2,WETH,18,4639646061444369151,0xe0554a476a092703abdb3ef35c80e0d76d32939f,7596.942337,4.639646,1637.396956


In [698]:
# Validate grouping 

grouped_df_test = uni_temp_df.groupby(['swaps_timestamp', 'swaps_tokenIn_id', 'swaps_tokenOut_id'])
group_sizes = grouped_df_test.size()
print(group_sizes)

swaps_timestamp  swaps_tokenIn_id                            swaps_tokenOut_id                         
1677410507       0xa0b86991c6218b36c1d19d4a2e9eb0ce3606eb48  0x30d20208d987713f46dfd34ef128bb16c404d10f    1
1677415223       0x375abb85c329753b1ba849a601438ae77eec9893  0xc02aaa39b223fe8d0a0e5c4f27ead9083c756cc2    1
1677416447       0x2b591e99afe9f32eaa6214f7b7629768c40eeb39  0xa0b86991c6218b36c1d19d4a2e9eb0ce3606eb48    1
1677418679       0xc02aaa39b223fe8d0a0e5c4f27ead9083c756cc2  0xdac17f958d2ee523a2206206994597c13d831ec7    2
1677418895       0x5283d291dbcf85356a21ba090e6db59121208b44  0xa0b86991c6218b36c1d19d4a2e9eb0ce3606eb48    1
                                                                                                          ..
1677482663       0xa0b86991c6218b36c1d19d4a2e9eb0ce3606eb48  0xc02aaa39b223fe8d0a0e5c4f27ead9083c756cc2    1
1677482771       0x2b591e99afe9f32eaa6214f7b7629768c40eeb39  0xa0b86991c6218b36c1d19d4a2e9eb0ce3606eb48    1
1677483707       0xc0c29

In [729]:
# MAX price in block 

In [730]:
# Group and perform MAX operator to obtain highest value in univ3. (i.e. The maximum price in the block) 
grouped_df_max = uni_temp_df.groupby(['swaps_timestamp', 'swaps_tokenIn_id', 'swaps_tokenOut_id']).agg({'uni_in_out_ratio': 'max'})

In [731]:
grouped_df_max.shape

(83, 1)

In [732]:
grouped_df_max.sort_values(by='swaps_timestamp', ascending=False, inplace=True)

In [735]:
grouped_df_max = grouped_df_max.rename(columns={'uni_in_out_ratio': 'univ3_blockMax'})

In [736]:
grouped_df_max

,,,univ3_blockMax
swaps_timestamp,swaps_tokenIn_id,swaps_tokenOut_id,
1677487391,0x5283d291dbcf85356a21ba090e6db59121208b44,0xa0b86991c6218b36c1d19d4a2e9eb0ce3606eb48,1.183797
1677484151,0xdac17f958d2ee523a2206206994597c13d831ec7,0x6b175474e89094c44da98b954eedeac495271d0f,0.999753
1677483707,0xc0c293ce456ff0ed870add98a0828dd4d2903dbf,0xc02aaa39b223fe8d0a0e5c4f27ead9083c756cc2,479.905460
1677482771,0x2b591e99afe9f32eaa6214f7b7629768c40eeb39,0xa0b86991c6218b36c1d19d4a2e9eb0ce3606eb48,14.917318
1677482663,0xa0b86991c6218b36c1d19d4a2e9eb0ce3606eb48,0xc02aaa39b223fe8d0a0e5c4f27ead9083c756cc2,1637.396956
...,...,...,...
1677418895,0x5283d291dbcf85356a21ba090e6db59121208b44,0xa0b86991c6218b36c1d19d4a2e9eb0ce3606eb48,1.201441
1677418679,0xc02aaa39b223fe8d0a0e5c4f27ead9083c756cc2,0xdac17f958d2ee523a2206206994597c13d831ec7,0.000625
1677416447,0x2b591e99afe9f32eaa6214f7b7629768c40eeb39,0xa0b86991c6218b36c1d19d4a2e9eb0ce3606eb48,17.709597


In [737]:
# VWAP of block 

In [738]:
uni_temp_df2 = uni_temp_df

In [739]:
uni_temp_df2['weighted_ratio'] = uni_temp_df2['univ3_Out_amount'] * uni_temp_df2['uni_in_out_ratio']

In [740]:
uni_temp_df2.head(2)

,swaps_id,swaps_hash,swaps_protocol_id,swaps_blockNumber,swaps_timestamp,swaps_tokenIn_id,tokenIn_symbol,tokenIn_decimal,swaps_amountIn,swaps_tokenOut_id,tokenOut_symbol,tokenOut_decimal,swaps_amountOut,swaps_pool_id,univ3_In_amount,univ3_Out_amount,uni_in_out_ratio,weighted_ratio
98,0xb9d34721a0da9ec11019805740abe5304f16fec3087bde92d2876788b0d544bc-234,0xb9d34721a0da9ec11019805740abe5304f16fec3087bde92d2876788b0d544bc,0x1f98431c8ad98523631ae4a59f267346ea31f984,16718418,1677487391,0x5283d291dbcf85356a21ba090e6db59121208b44,BLUR,18,8013004625170824296499,0xa0b86991c6218b36c1d19d4a2e9eb0ce3606eb48,USDC,6,6768900881,0x92ab871abb9d567aa276b2ce58d0203d84e0181e,8013.004625,6768.900881,1.183797,8013.004625
0,0x472150c620c0773767e6c5d3e9cd2a92cf7e8eb2001fbea5a74c6b86e5be5421-382,0x472150c620c0773767e6c5d3e9cd2a92cf7e8eb2001fbea5a74c6b86e5be5421,0x1f98431c8ad98523631ae4a59f267346ea31f984,16718150,1677484151,0xdac17f958d2ee523a2206206994597c13d831ec7,USDT,6,30510612544,0x6b175474e89094c44da98b954eedeac495271d0f,DAI,18,30518143256958443375976,0x48da0965ab2d2cbf1c17c09cfb5cbe67ad5b1406,30510.612544,30518.143257,0.999753,30510.612544


In [741]:
# chose weights as swap amounts out as they include all data in integers. 
grouped_df_vwap = uni_temp_df.groupby(['swaps_timestamp', 'swaps_tokenIn_id', 'swaps_tokenOut_id']).agg({'univ3_Out_amount':'sum','weighted_ratio':'sum'})

In [755]:
grouped_df_vwap['univ3_vwap'] = grouped_df_vwap['weighted_ratio'] / grouped_df_vwap['univ3_Out_amount'] 

In [756]:
grouped_df_vwap.sort_values(by='swaps_timestamp', ascending=False, inplace=True)

In [757]:
grouped_df_vwap.shape

(83, 3)

In [758]:
grouped_df_vwap

,,,univ3_Out_amount,weighted_ratio,univ3_vwap
swaps_timestamp,swaps_tokenIn_id,swaps_tokenOut_id,,,
1677487391,0x5283d291dbcf85356a21ba090e6db59121208b44,0xa0b86991c6218b36c1d19d4a2e9eb0ce3606eb48,6768.900881,8013.004625,1.183797
1677484151,0xdac17f958d2ee523a2206206994597c13d831ec7,0x6b175474e89094c44da98b954eedeac495271d0f,30518.143257,30510.612544,0.999753
1677483707,0xc0c293ce456ff0ed870add98a0828dd4d2903dbf,0xc02aaa39b223fe8d0a0e5c4f27ead9083c756cc2,3.849402,1847.349108,479.90546
1677482771,0x2b591e99afe9f32eaa6214f7b7629768c40eeb39,0xa0b86991c6218b36c1d19d4a2e9eb0ce3606eb48,10201.441775,152178.154868,14.917318
1677482663,0xa0b86991c6218b36c1d19d4a2e9eb0ce3606eb48,0xc02aaa39b223fe8d0a0e5c4f27ead9083c756cc2,4.639646,7596.942337,1637.396956
...,...,...,...,...,...
1677418895,0x5283d291dbcf85356a21ba090e6db59121208b44,0xa0b86991c6218b36c1d19d4a2e9eb0ce3606eb48,7603.470282,9135.118539,1.201441
1677418679,0xc02aaa39b223fe8d0a0e5c4f27ead9083c756cc2,0xdac17f958d2ee523a2206206994597c13d831ec7,751.316673,0.469937,0.000625
1677416447,0x2b591e99afe9f32eaa6214f7b7629768c40eeb39,0xa0b86991c6218b36c1d19d4a2e9eb0ce3606eb48,9899.264307,175311.977751,17.709597


#### Step 3: Merge cow trades with univ3_vwap and univ3_blockMax 

In [759]:
# merge trades and swaps on timestamp value, tokens, and direction of trade. We use outer join because we want to keep all trades and swaps data and backfill swap values
cow_univ3_merge1_df = pd.merge(cow_df, grouped_df_max, left_on=['trades_timestamp','trades_sellToken_id','trades_buyToken_id'], right_on=['swaps_timestamp','swaps_tokenIn_id','swaps_tokenOut_id'])

In [760]:
cow_univ3_merge1_df.sort_values(by='trades_timestamp', ascending=False, inplace=True)

In [761]:
cow_univ3_merge1_df.shape

(85, 19)

In [762]:
cow_univ3_merge1_df

,trades_id,trades_timestamp,trades_gasPrice,trades_feeAmount,trades_txHash,trades_settlement_id,trades_sellAmount,sell_token_decimal,trades_buyAmount,buy_token_decimal,trades_sellToken_id,trades_buyToken_id,trades_order_id,sell_token_symbol,buy_token_symbol,sell_amount,buy_amount,cow_sell_buy_ratio,univ3_blockMax
69,0x701a0176715a23bd53609b012f7bd58b67d503087c531df3d0d275cf6780820acbe203901e2f3ca910558efba39a48af89e3c55863fc73be|0xb9d34721a0da9ec11019805740abe5304f16fec3087bde92d2876788b0d544bc|121,1677487391,17033197478,10728549012072273920,0xb9d34721a0da9ec11019805740abe5304f16fec3087bde92d2876788b0d544bc,0xb9d34721a0da9ec11019805740abe5304f16fec3087bde92d2876788b0d544bc,80140774800720310238916,18,67751282514,6,0x5283d291dbcf85356a21ba090e6db59121208b44,0xa0b86991c6218b36c1d19d4a2e9eb0ce3606eb48,0x701a0176715a23bd53609b012f7bd58b67d503087c531df3d0d275cf6780820acbe203901e2f3ca910558efba39a48af89e3c55863fc73be,BLUR,USDC,80140.774801,67751.282514,1.182867,1.183797
0,0x80fcb44508ad58cc156422124f63ed9a9eb452030c45a20d07bee14e8727508f6d4ac258367dc87b219cd6025c6624199823528a63fd1718|0x472150c620c0773767e6c5d3e9cd2a92cf7e8eb2001fbea5a74c6b86e5be5421|176,1677484151,21094309984,0,0x472150c620c0773767e6c5d3e9cd2a92cf7e8eb2001fbea5a74c6b86e5be5421,0x472150c620c0773767e6c5d3e9cd2a92cf7e8eb2001fbea5a74c6b86e5be5421,30514028354,6,30518143245082152756689,18,0xdac17f958d2ee523a2206206994597c13d831ec7,0x6b175474e89094c44da98b954eedeac495271d0f,0x80fcb44508ad58cc156422124f63ed9a9eb452030c45a20d07bee14e8727508f6d4ac258367dc87b219cd6025c6624199823528a63fd1718,USDT,DAI,30514.028354,30518.143245,0.999865,0.999753
40,0xb73d730b3fb39d57ecb8fc58ed443fc1e7d95dff0a15e47ce49fdc020f7631fe36cc7b13029b5dee4034745fb4f24034f3f2ffc663fc7602|0x3df3c991cf0dc85a69c833862c66114a60a1e269d5f3a9ef745a41d16c76fbd2|92,1677483707,15858459108,3255068885491244032,0x3df3c991cf0dc85a69c833862c66114a60a1e269d5f3a9ef745a41d16c76fbd2,0x3df3c991cf0dc85a69c833862c66114a60a1e269d5f3a9ef745a41d16c76fbd2,6161085427427419246920,18,12823960269097420856,18,0xc0c293ce456ff0ed870add98a0828dd4d2903dbf,0xc02aaa39b223fe8d0a0e5c4f27ead9083c756cc2,0xb73d730b3fb39d57ecb8fc58ed443fc1e7d95dff0a15e47ce49fdc020f7631fe36cc7b13029b5dee4034745fb4f24034f3f2ffc663fc7602,AURA,WETH,6161.085427,12.82396,480.435474,479.905460
74,0x4f932d189954a3ed440d52840b058cd3da6465d4584bd9db7448aa792566a8cdbdc3922244608bdb27f619616e6aded5cc0c4e6a63fc61b8|0x151ed3ef9aabdbfeaa7d679deed658af23c0daf949d99a0542dcc819464090fc|161,1677482771,14915399317,13175076080,0x151ed3ef9aabdbfeaa7d679deed658af23c0daf949d99a0542dcc819464090fc,0x151ed3ef9aabdbfeaa7d679deed658af23c0daf949d99a0542dcc819464090fc,44761598529281,8,30000000000,6,0x2b591e99afe9f32eaa6214f7b7629768c40eeb39,0xa0b86991c6218b36c1d19d4a2e9eb0ce3606eb48,0x4f932d189954a3ed440d52840b058cd3da6465d4584bd9db7448aa792566a8cdbdc3922244608bdb27f619616e6aded5cc0c4e6a63fc61b8,HEX,USDC,447615.985293,30000.0,14.920533,14.917318
28,0xf1356cee8f424ecea9d6387870333cf4dafc244b7dbe956dc16de5b8491e6637582f09e431d249a986269a201ca5087dfe5a4efa63fc5f29|0x3f03fb68fe311cfe4d9d956115b27a84a54e592899fabc203d7a2a346d2855ba|119,1677482663,16673563590,0,0x3f03fb68fe311cfe4d9d956115b27a84a54e592899fabc203d7a2a346d2855ba,0x3f03fb68fe311cfe4d9d956115b27a84a54e592899fabc203d7a2a346d2855ba,7600000000,6,4639646061444369255,18,0xa0b86991c6218b36c1d19d4a2e9eb0ce3606eb48,0xc02aaa39b223fe8d0a0e5c4f27ead9083c756cc2,0xf1356cee8f424ecea9d6387870333cf4dafc244b7dbe956dc16de5b8491e6637582f09e431d249a986269a201ca5087dfe5a4efa63fc5f29,USDC,WETH,7600.0,4.639646,1638.055985,1637.396956
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
72,0x72acdf4ab7b4602c28e65c906b9fb98166ed3383843f0e783ae5b59052801a3443f1fa199539df692645d7af8c8ef3f63e526fd763fb63b8|0x4ac6f759c0f9759e1a1a614e549043b4077b4c31e4a56472570e77b494c5a04e|214,1677418895,18933065036,10324347935853905920,0x4ac6f759c0f9759e1a1a614e549043b4077b4c31e4a56472570e77b494c5a04e,0x4ac6f759c0f9759e1a1a614e549043b4077b4c31e4a56472570e7

In [780]:
# Perform the merge to include vwap result
cow_univ3_merge2_df = pd.merge(cow_univ3_merge1_df, grouped_df_vwap, 
                               left_on=['trades_timestamp', 'trades_sellToken_id', 'trades_buyToken_id'], 
                               right_on=['swaps_timestamp', 'swaps_tokenIn_id', 'swaps_tokenOut_id'], 
                               how='left')


In [781]:
cow_univ3_merge2_df.head(5)

,trades_id,trades_timestamp,trades_gasPrice,trades_feeAmount,trades_txHash,trades_settlement_id,trades_sellAmount,sell_token_decimal,trades_buyAmount,buy_token_decimal,trades_sellToken_id,trades_buyToken_id,trades_order_id,sell_token_symbol,buy_token_symbol,sell_amount,buy_amount,cow_sell_buy_ratio,univ3_blockMax,univ3_Out_amount,weighted_ratio,univ3_vwap
0,0x701a0176715a23bd53609b012f7bd58b67d503087c531df3d0d275cf6780820acbe203901e2f3ca910558efba39a48af89e3c55863fc73be|0xb9d34721a0da9ec11019805740abe5304f16fec3087bde92d2876788b0d544bc|121,1677487391,17033197478,10728549012072273920,0xb9d34721a0da9ec11019805740abe5304f16fec3087bde92d2876788b0d544bc,0xb9d34721a0da9ec11019805740abe5304f16fec3087bde92d2876788b0d544bc,80140774800720310238916,18,67751282514,6,0x5283d291dbcf85356a21ba090e6db59121208b44,0xa0b86991c6218b36c1d19d4a2e9eb0ce3606eb48,0x701a0176715a23bd53609b012f7bd58b67d503087c531df3d0d275cf6780820acbe203901e2f3ca910558efba39a48af89e3c55863fc73be,BLUR,USDC,80140.774801,67751.282514,1.182867,1.183797,6768.900881,8013.004625,1.183797
1,0x80fcb44508ad58cc156422124f63ed9a9eb452030c45a20d07bee14e8727508f6d4ac258367dc87b219cd6025c6624199823528a63fd1718|0x472150c620c0773767e6c5d3e9cd2a92cf7e8eb2001fbea5a74c6b86e5be5421|176,1677484151,21094309984,0,0x472150c620c0773767e6c5d3e9cd2a92cf7e8eb2001fbea5a74c6b86e5be5421,0x472150c620c0773767e6c5d3e9cd2a92cf7e8eb2001fbea5a74c6b86e5be5421,30514028354,6,30518143245082152756689,18,0xdac17f958d2ee523a2206206994597c13d831ec7,0x6b175474e89094c44da98b954eedeac495271d0f,0x80fcb44508ad58cc156422124f63ed9a9eb452030c45a20d07bee14e8727508f6d4ac258367dc87b219cd6025c6624199823528a63fd1718,USDT,DAI,30514.028354,30518.143245,0.999865,0.999753,30518.143257,30510.612544,0.999753
2,0xb73d730b3fb39d57ecb8fc58ed443fc1e7d95dff0a15e47ce49fdc020f7631fe36cc7b13029b5dee4034745fb4f24034f3f2ffc663fc7602|0x3df3c991cf0dc85a69c833862c66114a60a1e269d5f3a9ef745a41d16c76fbd2|92,1677483707,15858459108,3255068885491244032,0x3df3c991cf0dc85a69c833862c66114a60a1e269d5f3a9ef745a41d16c76fbd2,0x3df3c991cf0dc85a69c833862c66114a60a1e269d5f3a9ef745a41d16c76fbd2,6161085427427419246920,18,12823960269097420856,18,0xc0c293ce456ff0ed870add98a0828dd4d2903dbf,0xc02aaa39b223fe8d0a0e5c4f27ead9083c756cc2,0xb73d730b3fb39d57ecb8fc58ed443fc1e7d95dff0a15e47ce49fdc020f7631fe36cc7b13029b5dee4034745fb4f24034f3f2ffc663fc7602,AURA,WETH,6161.085427,12.82396,480.435474,479.905460,3.849402,1847.349108,479.90546
3,0x4f932d189954a3ed440d52840b058cd3da6465d4584bd9db7448aa792566a8cdbdc3922244608bdb27f619616e6aded5cc0c4e6a63fc61b8|0x151ed3ef9aabdbfeaa7d679deed658af23c0daf949d99a0542dcc819464090fc|161,1677482771,14915399317,13175076080,0x151ed3ef9aabdbfeaa7d679deed658af23c0daf949d99a0542dcc819464090fc,0x151ed3ef9aabdbfeaa7d679deed658af23c0daf949d99a0542dcc819464090fc,44761598529281,8,30000000000,6,0x2b591e99afe9f32eaa6214f7b7629768c40eeb39,0xa0b86991c6218b36c1d19d4a2e9eb0ce3606eb48,0x4f932d189954a3ed440d52840b058cd3da6465d4584bd9db7448aa792566a8cdbdc3922244608bdb27f619616e6aded5cc0c4e6a63fc61b8,HEX,USDC,447615.985293,30000.0,14.920533,14.917318,10201.441775,152178.154868,14.917318
4,0xf1356cee8f424ecea9d6387870333cf4dafc244b7dbe956dc16de5b8491e6637582f09e431d249a986269a201ca5087dfe5a4efa63fc5f29|0x3f03fb68fe311cfe4d9d956115b27a84a54e592899fabc203d7a2a346d2855ba|119,1677482663,16673563590,0,0x3f03fb68fe311cfe4d9d956115b27a84a54e592899fabc203d7a2a346d2855ba,0x3f03fb68fe311cfe4d9d956115b27a84a54e592899fabc203d7a2a346d2855ba,7600000000,6,4639646061444369255,18,0xa0b86991c6218b36c1d19d4a2e9eb0ce3606eb48,0xc02aaa39b223fe8d0a0e5c4f27ead9083c756cc2,0xf1356cee8f424ecea9d6387870333cf4dafc244b7dbe956dc16de5b8491e6637582f09e431d249a986269a201ca5087dfe5a4efa63fc5f29,USDC,WETH,7600.0,4.639646,1638.055985,1637.396956,4.639646,7596.942337,1637.396956


In [782]:
cow_univ3_merge2_df.shape

(85, 22)

In [784]:
unique_txns = cow_univ3_merge2_df['trades_id'].unique()

In [787]:
cow_univ3_merge2_df.to_csv('zero_arb_cow_univ3.csv')